In [17]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold

In [18]:
df=pd.read_json('../01_Preprocessing/First.json').sort_index()

In [19]:
df.head(2)

,country,countryCoded,degree,degreeCoded,eng,engCoded,fieldGroup,fund,gpaBachelors,gpaMasters,gre,highLevelBachUni,highLevelMasterUni,papersGLOB,papersIRAN,targetUni,uniAccepted,uniSelected,year
2,united kingdom,4,Doctorate,1,High,1,7,True,3.5,3.442986,False,False,False,4,8,"heriot watt , edinburgh",n/a,"heriot watt , edinburgh",2007
3,united states,0,Doctorate,1,VeryHigh,0,5,True,2.5,3.500000,True,True,False,8,10,northwestern,"epfl switzerland, northwestern",northwestern,2010


In [20]:
def mydist(x, y):
    return np.sum((x-y)**2)
def jaccard(a, b):
    intersection = float(len(set(a) & set(b)))
    union = float(len(set(a) | set(b)))
    return 1.0 - (intersection/union)

In [21]:
# http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html
    
dist=['braycurtis','canberra','chebyshev','cityblock','correlation','cosine','euclidean','dice','hamming','jaccard','kulsinski','matching','rogerstanimoto','russellrao','sokalsneath','yule']
algorithm=['ball_tree', 'kd_tree', 'brute']

## On country (only MS)

In [22]:
df.columns

Index([u'country', u'countryCoded', u'degree', u'degreeCoded', u'eng',
       u'engCoded', u'fieldGroup', u'fund', u'gpaBachelors', u'gpaMasters',
       u'gre', u'highLevelBachUni', u'highLevelMasterUni', u'papersGLOB',
       u'papersIRAN', u'targetUni', u'uniAccepted', u'uniSelected', u'year'],
      dtype='object')

In [23]:
oldDf=df.copy()
df=df[['countryCoded','degreeCoded','engCoded', 'fieldGroup','fund','gpaBachelors','gre', 'highLevelBachUni', 'papersGLOB','papersIRAN']]
df=df[df.degreeCoded==0]
del df['degreeCoded']
bestAvg=[]
for alg in algorithm:
    for dis in dist:
        k_fold = KFold(n=len(df), n_folds=5)
        scores = []
        try:
            clf = KNeighborsClassifier(n_neighbors=3, weights='distance',algorithm=alg, metric=dis)
        except Exception as err:
#             print(alg,dis,'err')
            continue
        for train_indices, test_indices in k_fold:
            xtr = df.iloc[train_indices,(df.columns != 'countryCoded')]
            ytr = df.iloc[train_indices]['countryCoded']
            xte = df.iloc[test_indices, (df.columns != 'countryCoded')]
            yte = df.iloc[test_indices]['countryCoded']
            clf.fit(xtr, ytr)
            ypred = clf.predict(xte)
            acc=accuracy_score(list(yte),list(ypred))
            scores.append(acc*100)
        print(alg,dis,np.average(scores))
        bestAvg.append(np.average(scores))
print('>>>>>>>Best: ',np.max(bestAvg))

('ball_tree', 'braycurtis', 28.238013838013835)
('ball_tree', 'canberra', 28.238990638990639)
('ball_tree', 'chebyshev', 25.4979242979243)
('ball_tree', 'cityblock', 27.782010582010582)
('ball_tree', 'euclidean', 27.838176638176641)
('ball_tree', 'dice', 29.549206349206351)
('ball_tree', 'hamming', 29.264143264143264)
('ball_tree', 'jaccard', 29.549206349206351)
('ball_tree', 'kulsinski', 29.492063492063494)
('ball_tree', 'matching', 29.549043549043546)
('ball_tree', 'rogerstanimoto', 29.549043549043546)
('ball_tree', 'russellrao', 27.441758241758244)
('ball_tree', 'sokalsneath', 29.549206349206351)
('kd_tree', 'chebyshev', 26.58168498168498)
('kd_tree', 'cityblock', 27.838502238502237)
('kd_tree', 'euclidean', 27.952136752136756)
('brute', 'braycurtis', 28.295482295482298)
('brute', 'canberra', 28.067887667887668)
('brute', 'chebyshev', 25.097924297924301)
('brute', 'cityblock', 27.325681725681726)
('brute', 'correlation', 26.585917785917786)
('brute', 'cosine', 26.642409442409445)
('

## On Fund (only MS)

In [46]:
bestAvg=[]
for alg in algorithm:
    for dis in dist:
        k_fold = KFold(n=len(df), n_folds=5)
        scores = []
        try:
            clf = KNeighborsClassifier(n_neighbors=3, weights='distance',algorithm=alg, metric=dis)
        except Exception as err:
            continue
        for train_indices, test_indices in k_fold:
            xtr = df.iloc[train_indices, (df.columns != 'fund')]
            ytr = df.iloc[train_indices]['fund']
            xte = df.iloc[test_indices, (df.columns != 'fund')]
            yte = df.iloc[test_indices]['fund']
            clf.fit(xtr, ytr)
            ypred = clf.predict(xte)
            acc=accuracy_score(list(yte),list(ypred))
            score=acc*100
            scores.append(score)
            if (len(bestAvg)>1) :
                if(score > np.max(bestAvg)) :
                    bestClf=clf
        bestAvg.append(np.average(scores))
        print (alg,dis,np.average(scores))
print('>>>>>>>Best: ',np.max(bestAvg))

('ball_tree', 'braycurtis', 75.753846153846155)
('ball_tree', 'canberra', 73.812779812779823)
('ball_tree', 'chebyshev', 73.643304843304847)
('ball_tree', 'cityblock', 76.382254782254776)
('ball_tree', 'euclidean', 75.983719983719965)
('ball_tree', 'dice', 70.734717134717144)
('ball_tree', 'hamming', 75.126414326414334)
('ball_tree', 'jaccard', 70.734717134717144)
('ball_tree', 'kulsinski', 70.791697191697196)
('ball_tree', 'matching', 70.791534391534398)
('ball_tree', 'rogerstanimoto', 70.791534391534398)
('ball_tree', 'russellrao', 58.922425722425729)
('ball_tree', 'sokalsneath', 70.734717134717144)
('kd_tree', 'chebyshev', 74.213431013431006)
('kd_tree', 'cityblock', 76.553846153846152)
('kd_tree', 'euclidean', 76.039886039886042)
('brute', 'braycurtis', 76.096052096052105)
('brute', 'canberra', 73.927065527065537)
('brute', 'chebyshev', 72.903540903540915)
('brute', 'cityblock', 76.894261294261298)
('brute', 'correlation', 75.524623524623536)
('brute', 'cosine', 74.84086284086284)


### Best : ('brute', 'cityblock', 76.894261294261298)

In [73]:
me=[1,2,0,2.5,False,False,1,0]

In [74]:
n=bestClf.kneighbors([me])
n

(array([[ 0.5,  0.5,  0.5]]), array([[ 5, 18, 20]], dtype=int64))

In [76]:
for i in n[1]:
    print(xtr.iloc[i])

    countryCoded  engCoded  fieldGroup  gpaBachelors    gre highLevelBachUni  \
18             1         1           2           2.0  False             True   
77             5         1           0           3.0  False             True   
84             2         1           0           2.0  False            False   

    papersGLOB  papersIRAN  
18           4           2  
77           1           3  
84           1           0  
